# Equilibirum

Now that we have a supply curve and a demand curve, we can put them together to determine equilibrium prices and quantities. To do so, we need to consider a number of different cases. 

## Perfect competition

A market is perfectly competitive if no single consumer or producer can influence prices on their own (we say that, in this case, there is no market power, or that everybody is a price taker). In practice, this is likely to happen if there are a large number of firms selling an identical product to a large number of consumers, in a market where everybody has perfect information (i.e. know prices everywhere), without any externalities (negative/positive effects of production/consumption that are not born by the producer/consumer). In this case, a producer or consumer cannot influence the price by producing or consuming more or less - everybody is such a small part of the market that this will not change anything. Can you think of example products where the market might approach perfect competition?

In this case, the equilibrium price and quantity can be found at the intersection of the supply and demand curve. It should be easy to see why: if a producer produces an incrementally small amount more, its costs (the height of the supply curve) will be higher than the price, so this is not worth doing. If a producer produces an incrementally small amount less, its lost revenues (the price) will be higher than the costs saved (the height of the supply curve).

In a perfectly competitive equilibrium, social surplus, or social welfare (the sum of producer and consumer surplus) will be maximised - i.e., even though everybody selfishly chooses how much to produce, the total value of the market to producer and consumer is at its optimum. Again, it should be easy to see why. If we produce more than the equilibrium, the costs of doing this (the height of the supply curve) exceeds the consumer's utility from the additional amount produced (the height of the demand curve). If we produce less, we can increase social welfare since the consumers' utility of getting an additional unit is still higher than the cost of producing this unit.

However, the relative magnitudes of consumer and producer surplus depend on the shapes of the demand and supply curves. In the diagram below, a perfectly competitive equilibrium is plotted (assuming linear curves). Experiment with the parameters. Which surplus is the greatest if demand is highly price sensitive (highly elastic)? What if demand is not very price sensitive (highly inelastic)?

In [1]:
# from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
import dash_bootstrap_components as dbc

app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div(
    [
        # html.H4("Budget constraint"),
        dcc.Graph(id="graph"),
        html.P("Select supply curve intercept:"),
        dcc.Slider(
            id="slider-supply-intercept",
            min=0,
            max=100,
            step=1,
            value=0,
            marks={i: str(i) for i in range(0, 201, 25)},
        ),
        html.P("Select supply curve slope:"),
        dcc.Slider(
            id="slider-supply-slope",
            min=0.5,
            max=5,
            step=0.10,
            value=2,
            marks={i: str(i) for i in range(0, 6, 2)},
        ),
        html.P("Select demand curve intercept:"),
        dcc.Slider(
            id="slider-demand-intercept",
            min=0,
            max=125,
            step=1,
            value=100,
            marks={i: str(i) for i in range(0, 201, 25)},
        ),
        html.P("Select demand curve slope:"),
        dcc.Slider(
            id="slider-demand-slope",
            min=-5,
            max=0.1,
            step=0.10,
            value=-3,
            marks={i: str(i) for i in range(-5, 1, 1)},
        ),
    ]
)

@app.callback(
    Output("graph", "figure"),
    Input("slider-supply-intercept", "value"),
    Input("slider-supply-slope", "value"),
    Input("slider-demand-intercept", "value"),
    Input("slider-demand-slope", "value")
)

def plot(supply_intercept, supply_slope, demand_intercept, demand_slope):
    def supply_function(supply_slope, supply_intercept):
        x_ = np.arange(0.1, 100., 0.1)
        y_ = []
        for x in x_:
            y = supply_slope * x + supply_intercept
            y_.append(y)
        df_plot = pd.DataFrame({'Quantity': x_, 'Price': y_})
        return df_plot
    def demand_function(demand_slope, demand_intercept):
        x_ = np.arange(0.1, 100., 0.1)
        y_ = []
        for x in x_:
            y = demand_slope * x + demand_intercept
            y_.append(y)
        df_plot = pd.DataFrame({'Quantity': x_, 'Price': y_})
        return df_plot
    def quantity_calc(supply_slope, supply_intercept, demand_slope, demand_intercept):
        quantity = (demand_intercept - supply_intercept) / (supply_slope - demand_slope)
        return quantity
    def price_calc(quantity, supply_slope, supply_intercept):
        price = supply_slope * quantity + supply_intercept
        return price

    df_supply = supply_function(supply_slope, supply_intercept)
    fig1 = px.line(df_supply, x='Quantity', y='Price', range_x=[0,50], range_y=[0,100])
    fig1.update_traces(line_color='blue', name='Supply curve', showlegend=True)

    df_demand = demand_function(demand_slope, demand_intercept)
    fig2 = px.line(df_demand, x='Quantity', y='Price', range_x=[0,50], range_y=[0,100])
    fig2.update_traces(line_color='green', name='Demand curve', showlegend=True)
    
    fig = go.Figure(data=fig1.data + fig2.data)

    quantity = quantity_calc(supply_slope, supply_intercept, demand_slope, demand_intercept)
    price = price_calc(quantity, supply_slope, supply_intercept)

    producer_surplus = (price - supply_intercept) * quantity / 2
    consumer_surplus = (demand_intercept - price) * quantity / 2

    fig_shape1 = go.Figure(go.Scatter(x=[0,0,quantity], y=[demand_intercept,price,price], line=dict(color="LightGreen"), fill="toself"))
    consumer_surplus_text = "Consumer surplus = " + str(round(consumer_surplus, 2))
    fig_shape1.update_traces(name=consumer_surplus_text)

    fig_shape2 = go.Figure(go.Scatter(x=[0,0,quantity], y=[supply_intercept,price,price], line=dict(color="LightBlue"), fill="toself"))
    producer_surplus_text = "Producer surplus = " + str(round(producer_surplus, 2))
    fig_shape2.update_traces(name=producer_surplus_text)

    fig = go.Figure(data=fig.data + fig_shape1.data + fig_shape2.data, layout = fig1.layout)

    fig.add_shape(type="line", x0=0, y0=price, x1=quantity, y1=price, line=dict(color="red", width=3, dash="dash"))
    fig.add_shape(type="line", x0=quantity, y0=0, x1=quantity, y1=price, line=dict(color="grey", width=3, dash="dash"))

    return fig

app.run_server(mode='inline', port=8057, jupyter_height=800)

NB : very few markets are perfectly competitive. In practise, if there is no obviously large amount of market power (e.g. if the market has a significant number of suppliers and consumers) we often use the perfectly competitive model anyway, just because it is so convenient.

## Imperfect competition: Monopoly

A producer has a monopoly if it is the only one producing a particular product. Can you think of markets for which this is the case? In this case, the monopolist has market power (it is a price maker) because, if it increases or decreases its production, it changes prices. In particular, by decreasing the quantity it produces, it can create more scarcity, increasing the price. The equilibrium between supply and demand is now not at the intersection between the demand and supply curves. To see why, use the diagram below. See what happens if you increase or decrease the quantity produced. What is the profit (=producer surplus)-maximising quantity? How does this depend on the slopes of the demand and supply curves? NB: if you increase the monopoly quantity above the perfectly competitive equilibrium, visualising the producer surplus becomes difficult as there is a loss.

In [2]:
# from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
import dash_bootstrap_components as dbc

app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    dbc.Row([
        # html.H4("Budget constraint"),
        dcc.Graph(id="graph")
    ]),
    
    dbc.Row([
        html.P("Select supply curve intercept:"),
        dcc.Slider(
            id="slider-supply-intercept",
            min=0,
            max=100,
            step=1,
            value=0,
            marks={i: str(i) for i in range(0, 201, 25)}),
    ]),
    dbc.Row([
        html.P("Select supply curve slope:"),
        dcc.Slider(
            id="slider-supply-slope",
            min=0.5,
            max=5,
            step=0.10,
            value=2,
            marks={i: str(i) for i in range(0, 6, 2)})
    ]),
    dbc.Row([
        html.P("Select demand curve intercept:"),
        dcc.Slider(
            id="slider-demand-intercept",
            min=0,
            max=125,
            step=1,
            value=100,
            marks={i: str(i) for i in range(0, 201, 25)},)
    ]),
    dbc.Row([
        html.P("Select demand curve slope:"),
        dcc.Slider(
            id="slider-demand-slope",
            min=-5,
            max=0.1,
            step=0.10,
            value=-3,
            marks={i: str(i) for i in range(-5, 1, 1)})
    ]),
    dbc.Row([
        html.P("Select monopolist quantity:"),
        dcc.Slider(
            id="slider-monopolist-quantity",
            min=0,
            max=100,
            step=1,
            value=12,
            marks={i: str(i) for i in range(0, 101, 20)})
    ])
])

@app.callback(
    Output("graph", "figure"),
    Input("slider-supply-intercept", "value"),
    Input("slider-supply-slope", "value"),
    Input("slider-demand-intercept", "value"),
    Input("slider-demand-slope", "value"),
    Input("slider-monopolist-quantity", "value")
)

def plot(supply_intercept, supply_slope, demand_intercept, demand_slope, monopolist_quantity):
    def supply_function(supply_slope, supply_intercept):
        x_ = np.arange(0.1, 100., 0.1)
        y_ = []
        for x in x_:
            y = supply_slope * x + supply_intercept
            y_.append(y)
        df_plot = pd.DataFrame({'Quantity': x_, 'Price': y_})
        return df_plot
    def demand_function(demand_slope, demand_intercept):
        x_ = np.arange(0.1, 100., 0.1)
        y_ = []
        for x in x_:
            y = demand_slope * x + demand_intercept
            y_.append(y)
        df_plot = pd.DataFrame({'Quantity': x_, 'Price': y_})
        return df_plot
    def marginal_revenue(demand_slope, demand_intercept):
        x_ = np.arange(0.1, 100., 0.1)
        y_ = []
        for x in x_:
            y = 2 * demand_slope * x + demand_intercept
            y_.append(y)
        df_plot = pd.DataFrame({'Quantity': x_, 'Price': y_})
        return df_plot
    def producer_price_calc(monopolist_quantity, supply_slope, supply_intercept):
        price = supply_slope * monopolist_quantity + supply_intercept
        return price
    def demand_price_calc(monopolist_quantity, demand_slope, demand_intercept):
        price = demand_slope * monopolist_quantity + demand_intercept
        return price

    df_supply = supply_function(supply_slope, supply_intercept)
    fig1 = px.line(df_supply, x='Quantity', y='Price', range_x=[0,50], range_y=[0,100])
    fig1.update_traces(line_color='blue', name='Supply curve', showlegend=True)
    df_demand = demand_function(demand_slope, demand_intercept)
    fig2 = px.line(df_demand, x='Quantity', y='Price', range_x=[0,50], range_y=[0,100])
    fig2.update_traces(line_color='green', name='Demand curve', showlegend=True)
    df_marginal = marginal_revenue(demand_slope, demand_intercept)
    fig3 = px.line(df_marginal, x='Quantity', y='Price', range_x=[0,50], range_y=[0,100])
    fig3.update_traces(line_color='black', line_dash="dash", name='Marginal revenue', showlegend=True)
    fig = go.Figure(data=fig1.data + fig2.data + fig3.data, layout = fig1.layout)

    quantity = monopolist_quantity
    producer_price = producer_price_calc(monopolist_quantity, supply_slope, supply_intercept)
    demand_price = demand_price_calc(monopolist_quantity, demand_slope, demand_intercept)

    producer_surplus = quantity * (demand_price - producer_price) + (producer_price - supply_intercept) * quantity / 2
    consumer_surplus = (demand_intercept - demand_price) * quantity / 2

    fig_shape1 = go.Figure(go.Scatter(x=[0,0,quantity], y=[demand_intercept,demand_price,demand_price], line=dict(color="LightGreen"), fill="toself"))
    consumer_surplus_text = "Consumer surplus = " + str(round(consumer_surplus, 2))
    fig_shape1.update_traces(name=consumer_surplus_text)

    fig_shape2 = go.Figure(go.Scatter(x=[0,0,quantity,quantity], y=[supply_intercept,demand_price,demand_price,producer_price], line=dict(color="LightBlue"), fill="toself"))
    producer_surplus_text = "Producer surplus = " + str(round(producer_surplus, 2))
    fig_shape2.update_traces(name=producer_surplus_text)

    fig = go.Figure(data=fig.data + fig_shape1.data + fig_shape2.data, layout = fig1.layout)

    # fig.add_shape(type="line", x0=0, y0=price, x1=quantity, y1=price, line=dict(color="red", width=3, dash="dash"))
    fig.add_shape(type="line", x0=quantity, y0=0, x1=quantity, y1=demand_price, line=dict(color="LightGrey", width=3, dash="dash"))

    return fig

app.run_server(mode='inline', port=8058, jupyter_height=950)

You have (hopefully) seen that the monopolist would choose a quantity where the supply curve intersects the blue dot-dashed line. This is the marginal revenue function which, when supply and demand curves are linear, is twice as steep as the demand curve. To see where this comes from, we have to do a bit of mathematics. A monopolist sets a quantity such that its marginal revenue (the revenue from producing an additional unit) is equal to its marginal cost (the cost of producing an additional unit). If these two are not equal, it can still increase its profits. Marginal costs are, as we have seen, given by the supply curve. Total revenues (the amount of money coming in) are (price × quantity) where the price is given by the inverse demand function (price = demand intercept + demand slope × quantity). Putting these two together, total revenues are given by TR = price × quantity = (demand intercept + demand slope x quantity) × quantity = demand intercept × quantity + demand slope × quantity2 Hence, marginal revenues (the additional amount of money coming in if the producer makes another unit) are D[TR]
quantity = demand intercept + 2 × demand slope × quantity.

Note that, as we know, the demand slope is generally negative. This is the marginal revenue function plotted above. If a monopolist produces another unit, it gets the price for this additinal unit (demand intercept + demand slope x quantity) but this also reduces the price it gets for all its products (otherwise the monopolist would not sell the additional unit).

Knowing this, go back to the diagram. For which demand/supply slopes/intercepts do consumers suffer most/least from being in a monopolistic market, as opposed to the perfectly competitive market above? What would happen if we shift the supply curve up or down in a parallel way (e.g., if we gave the producer a per-unit subsidy or imposed a tax)? For which parameters would consumers/the monopolist benefit most? Remember to optimise the quantity every time you change the intercepts/slopes. What if we shift the demand curve up or down (e.g, by increasing or reducing value-added taxes)?

## Imperfect competition: Oligopoly

Most markets are neither perfectly competitive nor monopolies. Instead, there are usually a number (but not an infinite number) of firms that supply a product. In this case, the market is an oligopoly. At this point, we have to consider the nature of competition. In some markets, producers can be assumed to choose prices. Quantities are then a consequence of the prices that everybody charges, as well as the demand curve. This is called Bertrand competition. In other markets, it is more natural to think of producers as choosing quantities. In this case, prices are then a consequence of the quantities all producers choose, and the demand curve. This is called Cournot competition. An often-cited example are air travel markets, where the important initial choice facing airlines is the number of seats they will offer in a market; after all airlines have chosen this, a market-clearing price can be calculated. Energy markets are also usually thought of as Cournot markets. Finally, sometimes some producers are market leaders, who get to make their decisions before others, instead of everybody making decisions at the same time. In this case, we have Stackelberg competition.

### Bertrand oligopoly (price competition)

Bertrand oligopolies are easy to model. As long as consumers have perfect information about the prices that all producers charge, they will buy from the producer with the lowest price. This means that a producer can capture the entire market by choosing a price that is just a little bit lower than its competitors. Of course, its competitors know this, so will also try to undercut. This leads to a race to the bottom. Producers will not be willing to sell products below their costs, so a producer will set its production to zero if it reaches its cost of production. Once there are only two producers left, with one having reached its cost of production, the second one has no incentive to further reduce its price. The price in a Bertrand oligopoly is therefore the production cost of the second-cheapest producer. All remaining producers will charge this price. If all producers have a constant marginal cost, only two producers will produce. If both producers have equal marginal costs, the price will be equal to what it would be under perfect competition. If producers have quadratic total cost curves (i.e. linear marginal cost curves) or above, more than two producers might be active. Since energy markets are not usually modelled as Bertrand oligopolies, we will not analyse this setting in detail.

### Cournot oligopoly (quantity competition)

In a Cournot oligopoly, all producers choose quantities. For simplicity, we will analyse a situation with only two producers, i.e. a Cournot duopoly. Once we get to energy market examples, we will consider settings with more producers. Suppose that we have two producers with linear marginal cost functions:
cost 1 = cost intercept 1 + cost slope 1 × quantity 1
cost 2 = cost intercept 2 + cost slope 2 × quantity 2

and an inverse demand function

price = demand intercept + demand slope × total quantity = demand intercept + demand slope × (quantity 1 + quantity 2)
(Note that the demand slope is usually negative.)

Profits are then given by 

profit 1 = price × quantity 1 - cost 1 = (demand intercept + demand slope × (quantity 1 + quantity 2)) × quantity 1 - (cost intercept 1 + cost slope 1 × quantity 1)
profit 2 = price × quantity 2 - cost 2 = (demand incercept + demand slope × (quantity 1 + quantity 2)) × quantity 2 - (cost intercept 2 + cost slope 2 × quantity 2)

Each firm’s profits therefore depend not only on what it does, but also on what its competitor does. Each firm optimises its quantity, so
∂ profit 1
∂ quantity 1 = ∂ profit 2
∂ quantity 2 = 0. Taking the derivatives gives:
demand intercept - cost slope 1 + 2 × demand slope × quantity 1 + demand slope × quantity 2 = 0
demand intercept - cost slope 2 + 2 × demand slope × quantity 2 + demand slope × quantity 1 = 0

We could solve these directly to obtain optimal quantities. We can also rearrange these equations to obtain optimal response functions, i.e., the optimal quantity that each producer should produce, given its competitors quantity choice:
optimal quantity 1 = cost slope 1 - demand intercept - demand slope × quantity 2
2 × demand slope

optimal quantity 2 = cost slope 2 - demand intercept - demand slope × quantity 1
2 × demand slope

We can visualise this in the diagram below. On producer 1’s optimal response function, we can read how much it should produce given a known quantity 2. On producer 2’s optimal response function, we can read how much it should produce given a known quantity 1.

In [3]:
# from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
import dash_bootstrap_components as dbc

app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div(
    [
        # html.H4("Budget constraint"),
        dcc.Graph(id="graph"),
        html.P("Select demand intercept:"),
        dcc.Slider(
            id="slider-demand-intercept",
            min=0,
            max=200,
            step=1,
            value=200,
            marks={i: str(i) for i in range(0, 201, 25)},
        ),
        html.P("Select demand slope:"),
        dcc.Slider(
            id="slider-demand-slope",
            min=-5,
            max=0.1,
            step=0.10,
            value=-0.2,
            marks={i: str(i) for i in range(-5, 1, 1)},
        ),
        # html.P("Select cost intercept 1:"),
        # dcc.Slider(
        #     id="slider-cost-intercept-1",
        #     min=0,
        #     max=100,
        #     step=0.5,
        #     value=2.5,
        #     marks={i: str(i) for i in range(0, 101, 25)},
        # ),
        # html.P("Select cost intercept 2:"),
        # dcc.Slider(
        #     id="slider-cost-intercept-2",
        #     min=0,
        #     max=100,
        #     step=0.5,
        #     value=2.5,
        #     marks={i: str(i) for i in range(0, 101, 25)},
        # ),
        html.P("Select cost slope 1:"),
        dcc.Slider(
            id="slider-cost-slope-1",
            min=0,
            max=100,
            step=0.5,
            value=0.5,
            marks={i: str(i) for i in range(0, 101, 25)},
        ),
        html.P("Select cost slope 2:"),
        dcc.Slider(
            id="slider-cost-slope-2",
            min=0,
            max=100,
            step=0.5,
            value=0.5,
            marks={i: str(i) for i in range(0, 101, 25)},
        ),
    ]
)

@app.callback(
    Output("graph", "figure"),
    Input("slider-demand-intercept", "value"),
    Input("slider-demand-slope", "value"),
    # Input("slider-cost-intercept-1", "value"),
    # Input("slider-cost-intercept-2", "value"),
    Input("slider-cost-slope-1", "value"),
    Input("slider-cost-slope-2", "value"),
)

def plot(demand_intercept, demand_slope, cost_slope_1, cost_slope_2):
    def optimal_response_function_1(cost_slope, demand_intercept, demand_slope):
        x_ = np.arange(0.1, 1000., 0.1)
        y_ = []
        for x in x_:
            y = (cost_slope - demand_intercept - demand_slope * x) / (2 * demand_slope)
            y_.append(y)
        df_plot = pd.DataFrame({'Quantity 2': x_, 'Quantity 1': y_})
        return df_plot
    def optimal_response_function_2(cost_slope, demand_intercept, demand_slope):
        y_ = np.arange(0.1, 1000., 0.1)
        x_ = []
        for y in y_:
            x = (cost_slope - demand_intercept - demand_slope * y) / (2 * demand_slope)
            x_.append(x)
        df_plot = pd.DataFrame({'Quantity 2': x_, 'Quantity 1': y_})
        return df_plot

    df_optimal_response_function_1 = optimal_response_function_1(cost_slope_1, demand_intercept, demand_slope)
    fig1 = px.line(df_optimal_response_function_1, x='Quantity 2', y='Quantity 1', range_x=[0,500], range_y=[0,1000])
    fig1.update_traces(line_color='blue', name='Optimal response function 1', showlegend=True)

    df_optimal_response_function_2 = optimal_response_function_2(cost_slope_2, demand_intercept, demand_slope)
    fig2 = px.line(df_optimal_response_function_2, x='Quantity 2', y='Quantity 1', range_x=[0,500], range_y=[0,1000])
    fig2.update_traces(line_color='green', name='Optimal response function 2', showlegend=True)
    
    fig = go.Figure(data=fig1.data + fig2.data, layout = fig1.layout)

    return fig

app.run_server(mode='inline', port=8059, jupyter_height=800)

The equilibrium will be at the intersection of the two response functions, as only there, none of the two producers has an incentive to choose anything else. The diagram below shows this in more detail. This is a cobweb diagram. In this diagram, you can see what would happen if producer 1 starts out selecting a particular quantity. Producer 2 will respond to this by choosing its optimal response. Seeing this, producer 1 will change its quantity to the new optimal response to producer 2, and so on. As you can see, any starting value will eventually lead to you end up at the intersection.

The intersection is, of course, the point where ∂ profit 1
∂ quantity 1 = ∂ profit 2
∂ quantity 2 = 0, so you can also find this point directly, but I wanted to show you the cobweb diagram to get some insight into why you might expect to see this equilibrium.

### Stackelberg competition

In Stackelberg competition, one of the producers gets to make a decision first. This could be a price decision or a quantity decision. If the decision is a price decision, the outcome of very similar to Bertrand competition: the first mover will anticipate possible undercutting, and set a price such that its competitors cannot undercut it, i.e. it will set its price to the marginal cost of its cheapest competitor (or choose to not participate in the market at all, if its own costs are higher).

Stackelberg competition in quantities is more interesting. It is perhaps not so often used in short-term energy market applications, where all producers make decisions at the same time, but is very useful to model investment, as there, some firms are clear leaders and others will only choose to invest once they have seen what the leaders are going to do. Note that it is not necessarily an advantage to be the leader (i.e. there is not necessary a first-mover advantage); it might be better to be able to wait and see, depending on the setting.

Let’s look at a very quick example, taking the same two profit functions and the demand function from the Cournot example above:

profit 1 = price × quantity 1 - cost 1 = (demand intercept + demand slope × (quantity 1 + quantity 2)) × quantity 1 - (cost intercept 1 + cost slope 1 × quantity 1)

profit 2 = price × quantity 2 - cost 2 = (demand incercept + demand slope × (quantity 1 + quantity 2)) × quantity 2 - (cost intercept 2 + cost slope 2 × quantity 2)

As before, each firm’s profits therefore depend not only on what it does, but also on what its competitor does. However, let us now assume that the first firm gets to choose its quantity first. That means that, when the second firm is deciding, it already knows quantity 1. As before, it will set ∂ profit 2
∂ quantity 2 = 0, and therefore
optimal quantity 2 = cost slope 2 - demand intercept - demand slope × quantity 1
2 × demand slope

The first producer knows that this is what his follower will do. It gets to decide first, so it can plug the above quantity into its own profit function and then determine what its optimal quantity as leader should be:

profit 1 = (demand intercept + demand slope × (quantity 1 + cost slope 2 - demand intercept - demand slope × quantity 12 × demand slope )) × quantity 1 - (cost intercept 1 + cost slope 1 ×
quantity 1)

To determine the optimum, we need ∂ profit 1 ∂ quantity 1 optimal quantity 2 = 0, i.e. the derivative of the above profit function is equal to zero. I will not do these derivations here (you can do them yourself if you want to get some practice), but you can use the diagram below to see the difference between the Cournot and Stackelberg equilibria. Which situation would you prefer if you were producer 1 or 2? How does this depend on the model parameters? Can you explain why?

In [4]:
# Stackelberg # from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
import plotly.express as px

import numpy as np
import pandas as pd
import dash_bootstrap_components as dbc

app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            dcc.Graph(id="graph"),
        ]),
        dbc.Col([
            dcc.Graph(id="graph2"),
        ]),
        dbc.Col([
            dcc.Graph(id="graph3"),
        ]),
    ]),
    dbc.Row([
        html.P("Select demand intercept:"),
        dcc.Slider(
            id="slider-demand-intercept",
            min=0,
            max=200,
            step=1,
            value=200,
            marks={i: str(i) for i in range(0, 201, 25)},
        ),
    ]),
    dbc.Row([
        html.P("Select demand slope:"),
        dcc.Slider(
            id="slider-demand-slope",
            min=-5,
            max=0.1,
            step=0.10,
            value=-0.2,
            marks={i: str(i) for i in range(-5, 1, 1)},
        ),
    ]),
    dbc.Row([
        html.P("Select cost intercept 1:"),
        dcc.Slider(
            id="slider-cost-intercept-1",
            min=0,
            max=100,
            step=0.5,
            value=2.5,
            marks={i: str(i) for i in range(0, 101, 25)},
        ),
    ]),
    dbc.Row([
        html.P("Select cost intercept 2:"),
        dcc.Slider(
            id="slider-cost-intercept-2",
            min=0,
            max=100,
            step=0.5,
            value=2.5,
            marks={i: str(i) for i in range(0, 101, 25)},
        ),
    ]),
    dbc.Row([
        html.P("Select cost slope 1:"),
        dcc.Slider(
            id="slider-cost-slope-1",
            min=0,
            max=100,
            step=0.5,
            value=0.5,
            marks={i: str(i) for i in range(0, 101, 25)},
        ),
    ]),
    dbc.Row([
        html.P("Select cost slope 2:"),
        dcc.Slider(
            id="slider-cost-slope-2",
            min=0,
            max=100,
            step=0.5,
            value=0.5,
            marks={i: str(i) for i in range(0, 101, 25)},
        ),
    ]),
    ]
)

@app.callback(
    Output("graph", "figure"),
    Output("graph2", "figure"),
    Output("graph3", "figure"),
    Input("slider-demand-intercept", "value"),
    Input("slider-demand-slope", "value"),
    Input("slider-cost-intercept-1", "value"),
    Input("slider-cost-intercept-2", "value"),
    Input("slider-cost-slope-1", "value"),
    Input("slider-cost-slope-2", "value"),
)

def plot(demand_intercept, demand_slope, cost_intercept_1, cost_intercept_2, cost_slope_1, cost_slope_2):
    def stack_quantity_1(cost_slope_1, cost_slope_2, demand_intercept, demand_slope):
        q1 = (2 * cost_slope_1 - demand_intercept - cost_slope_2) / (2 * demand_slope)
        return q1
    def stack_quantity_2(cost_slope_2, demand_intercept, demand_slope, q1):
        q2 = (cost_slope_2 - demand_intercept - demand_slope * q1) / (2 * demand_slope)
        return q2
    def stack_profit_1(demand_intercept, demand_slope, stack_q1, stack_q2, cost_intercept_1, cost_slope_1):
        profit1 = demand_intercept * stack_q1 + demand_slope * stack_q1 * stack_q2 + demand_slope * stack_q1 ** 2 - cost_intercept_1 - cost_slope_1 * stack_q1
        return profit1
    def stack_profit_2(demand_intercept, demand_slope, stack_q1, stack_q2, cost_intercept_2, cost_slope_2):
        profit2 = demand_intercept * stack_q2 + demand_slope * stack_q2 * stack_q1 + demand_slope * stack_q2 ** 2 - cost_intercept_2 - cost_slope_2 * stack_q2
        return profit2
    def stack_price(demand_intercept, demand_slope, stack_q1, stack_q2):
        price1 = demand_intercept + demand_slope * (stack_q1 + stack_q2)
        return price1

    def cournot_quantity_1(cost_slope_1, cost_slope_2, demand_slope):
        q1 = (2 * cost_slope_1 - cost_slope_2 - demand_intercept) / (3 * demand_slope)
        return q1
    def cournot_quantity_2(cost_slope_2, demand_slope, demand_intercept, q1):
        q2 = (cost_slope_2 - demand_intercept - demand_slope * q1) / (2 * demand_slope)
        return q2
    def cournot_profit_1(demand_intercept, demand_slope, cournot_q1, cournot_q2, cost_intercept_1, cost_slope_1):
        profit1 = demand_intercept * cournot_q1 + demand_slope * cournot_q1 * cournot_q2 + demand_slope * cournot_q1 ** 2 - cost_intercept_1 - cost_slope_1 * cournot_q1
        return profit1
    def cournot_profit_2(demand_intercept, demand_slope, cournot_q1, cournot_q2, cost_intercept_2, cost_slope_2):
        profit2 = demand_intercept * cournot_q2 + demand_slope * cournot_q2 * cournot_q1 + demand_slope * cournot_q2 ** 2 - cost_intercept_2 - cost_slope_2 * cournot_q2
        return profit2
    def cournot_price(demand_intercept, demand_slope, cournot_q1, cournot_q2):
        price1 = demand_intercept + demand_slope * (cournot_q1 + cournot_q2)
        return price1
    
    stack_q1 = stack_quantity_1(cost_slope_1, cost_slope_2, demand_intercept, demand_slope)
    stack_q2 = stack_quantity_2(cost_slope_2, demand_intercept, demand_slope, stack_q1)
    stack_profit1 = stack_profit_1(demand_intercept, demand_slope, stack_q1, stack_q2, cost_intercept_1, cost_slope_1)
    stack_profit2 = stack_profit_2(demand_intercept, demand_slope, stack_q1, stack_q2, cost_intercept_2, cost_slope_2)
    stack_pri = stack_price(demand_intercept, demand_slope, stack_q1, stack_q2)

    cournot_q1 = cournot_quantity_1(cost_slope_1, cost_slope_2, demand_slope)
    cournot_q2 = cournot_quantity_2(cost_slope_2, demand_slope, demand_intercept, cournot_q1)
    cournot_profit1 = cournot_profit_1(demand_intercept, demand_slope, cournot_q1, cournot_q2, cost_intercept_1, cost_slope_1)
    cournot_profit2 = cournot_profit_2(demand_intercept, demand_slope, cournot_q1, cournot_q2, cost_intercept_2, cost_slope_2)
    cournot_pri = cournot_price(demand_intercept, demand_slope, cournot_q1, cournot_q2)
    
    game = ['Stackelberg', 'Cournot']

    fig1 = go.Figure(data=[
        go.Bar(name='Q1', x=game, y=[stack_q1, cournot_q1]),
        go.Bar(name='Q2', x=game, y=[stack_q2, cournot_q2])
    ])
    fig1.update_layout(title_text='Quantity', showlegend=False, margin=dict(l=20, r=20))

    fig2 = go.Figure(data=[
        go.Bar(name='Price', x=game, y=[stack_pri, cournot_pri])
    ])
    fig2.update_layout(title_text='Price', margin=dict(l=20, r=20))

    fig3 = go.Figure(data=[
        go.Bar(name='Profit 1', x=game, y=[stack_profit1, cournot_profit1]),
        go.Bar(name='Profit 2', x=game, y=[stack_profit2, cournot_profit2])
    ])
    fig3.update_layout(title_text='Profit', showlegend=False, margin=dict(l=20, r=20))

    return fig1, fig2, fig3

app.run(mode='inline', port=8060, jupyter_height=1000)

## Imperfect competition: Consumer market power

Consumers can also have market power. This is less common, but there are plenty of examples of markets with only a small number of large consumers. Think, for instance, about markets for specialist medical equipment, computing parts, or manufacturing equipment. If there is only one consumer for a product (but potentially a large number of producers) this is called a monopsony. If there are several, this is an oligopsoly. Since these are not very common, and have less relevance to energy/environmental markets, we will not discuss them in detail here.


## A note on short vs. long-term decisions

So far, we have only looked at short-term decision making (i.e. assuming that the number of competitors is fixed). In reality, producers can usually choose to leave a market if they are making losses. Similarly, producers can enter if there are profits to be made, although there might be barriers to entry/exit, such as high fixed costs involved in setting yourself up as a new producer, or regulatory constraints. Hence, monopolies might evolve into oligopolies or perfectly competitive markets. Long-run profits are equal to the short-term producer profits minus fixes costs. We will discuss this later when we look at energy market examples.